In [25]:
import pandas as pd
from google.colab import drive
import os
from zipfile import ZipFile
import numpy as np
import json


drive.mount('/content/drive')

testgenes = pd.read_csv("top_interaction_sequences.csv")

colabpath = r"/content"

AF_outputs = r"/content/drive/MyDrive/Fitness_Map_Group_Project/Alphafold_2_outputs"   #Replace with the path to the Alphafold_2_outputs folder for your mounted drive. Shared folder is here https://drive.google.com/drive/folders/1Gz7YlvsNBLRM48aVSyd6X7ODeR3TPklX?usp=drive_link

testgenes["Min_PAE_of_Interaction"] = float("nan")
testgenes["Mean_PAE_of_Interaction"] = float("nan")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
def get_pae(chainA_len, chainB_len):
  # Assume residues from chain A are from i to j, and chain B from k to l
  # For example:
  chain_A_range = range(0, chainA_len+1)     # residue indices for chain A
  chain_B_range = range(chainA_len+1, chainA_len + chainB_len + 1)   # residue indices for chain B

  # Extract the inter-chain PAE block
  inter_chain_pae = pae[np.ix_(chain_A_range, chain_B_range)]

  # Compute metrics
  min_pae = np.min(inter_chain_pae)
  mean_pae = np.mean(inter_chain_pae)

  print(f"Minimum PAE between chains: {min_pae:.2f}")
  print(f"Mean PAE between chains: {mean_pae:.2f}")

  return min_pae, mean_pae

In [27]:
for folder in os.listdir(AF_outputs):
  if ".zip" in folder:
    with ZipFile((os.path.join(AF_outputs,folder)), 'r') as zObject:

    # Extracting all the members of the zip into google colab
      zObject.extractall(
          path="/content")
    gene_pair = folder[:folder.find(".result.zip")]
    first_ = gene_pair.find("_")
    second_ = gene_pair.find("_",first_+1)
    third_ = gene_pair.find("_",second_+1)
    gene1 = int(gene_pair[first_+1:second_])
    gene2 = int(gene_pair[second_+1:third_])
    print(gene1,gene2)

    pae_file = gene_pair + "_predicted_aligned_error_v1.json"
    with open("/content/" + os.path.join(gene_pair, pae_file)) as f:
      pae_data = json.load(f)
    pae = np.array(pae_data['predicted_aligned_error'])

    #Note:
    # pae.shape
    # len(testgenes.loc[0,"gene1_seq"]) + len(testgenes.loc[0,"gene2_seq"])  #the shape of the pae array is the length of the sequences added together

    gene_row = testgenes[(testgenes["gene1_locusId"] == gene1) & (testgenes["gene2_locusId"] == gene2)]
    gene1_seq = gene_row["gene1_seq"]
    gene2_seq = gene_row["gene2_seq"]

    min_pae, mean_pae = get_pae(len(gene1_seq), len(gene2_seq))

    testgenes.loc[gene_row.index,"Min_PAE_of_Interaction"] = min_pae

    testgenes.loc[gene_row.index,"Mean_PAE_of_Interaction"] = mean_pae



18059 17626
Minimum PAE between chains: 2.13
Mean PAE between chains: 3.05
17552 18130
Minimum PAE between chains: 1.67
Mean PAE between chains: 2.59
17245 15651
Minimum PAE between chains: 1.63
Mean PAE between chains: 3.08
17552 17261
Minimum PAE between chains: 1.52
Mean PAE between chains: 2.33
17552 16347
Minimum PAE between chains: 1.72
Mean PAE between chains: 2.36
17552 15217
Minimum PAE between chains: 1.48
Mean PAE between chains: 2.28


In [28]:
testgenes

,Unnamed: 0,gene1,gene1_locusId,gene1_seq,gene2,gene2_locusId,gene2_seq,coefficient,Min_PAE_of_Interaction,Mean_PAE_of_Interaction
0,9725021,Gene_2567,17245,MTVFNKFARTFKSHWLLYLCVIVFGITNLVASSGAHMVQRLLFFVL...,Gene_1225,15651,MKSTSDLFNEIIPLGRLIHMVNQKKDRLLNEYLSPLDITAAQFKVL...,5.033844,1.63,3.085
1,10544578,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_2574,17261,MENNEIQSVLMNALSLQEVHVSGDGSHFQVIAVGELFDGMSRVKKQ...,4.562073,1.52,2.325
2,10543821,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_1817,16347,MLSIFKPAPHKARLPAAEIDPTYRRLRWQIFLGIFFGYAAYYLVRK...,4.355901,1.72,2.365
3,12025962,Gene_3174,18059,MNTQYNSSYIFSITLVATLGGLLFGYDTAVISGTVESLNTVFVAPQ...,Gene_2850,17626,MQAYFDQLDRVRYEGSKSSNPLAFRHYNPDELVLGKRMEEHLRFAA...,4.136822,2.13,3.050
4,10545240,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_3237,18130,MHLSTHPTSYPTRYQEIAAKLEQELRQHYRCGDYLPAEQQLAARFE...,4.050859,1.67,2.595
5,12025961,Gene_3174,18059,MNTQYNSSYIFSITLVATLGGLLFGYDTAVISGTVESLNTVFVAPQ...,Gene_2849,17625,MYIGIDLGTSGVKVILLNEQGEVVAAQTEKLTVSRPHPLWSEQDPE...,3.873120,NaN,NaN
6,10542883,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_879,15217,MSKRRVVVTGLGMLSPVGNTVESTWKALLAGQSGISLIDHFDTSAY...,3.862169,1.48,2.280
7,10544985,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_2982,17788,MKVMRTTVATVVAATLSMSAFSVFAEASLTGAGATFPAPVYAKWAD...,3.846149,NaN,NaN
8,10542110,Gene_2783,17552,MNIYIGWLFKLIPLIMGLICIALGGFVLESSGQSEYFVAGHVLISL...,Gene_106,14288,MTVAYIAIGSNLASPLEQVNAALKALGDIPESHILTVSSFYRTPPL...,3.721365,NaN,NaN
9,10806226,Gene_2852,17628,MPYLLEMKNITKTFGSVKAIDNVCLRLNAGEIVSLCGENGSGKSTL...,Gene_2850,17626,MQAYFDQLDRVRYEGSKSSNPLAFRHYNPDELVLGKRMEEHLRFAA...,3.665344,NaN,NaN


In [29]:
testgenes.isnull().sum()

,0
Unnamed: 0,0
gene1,0
gene1_locusId,0
gene1_seq,0
gene2,0
gene2_locusId,0
gene2_seq,0
coefficient,0
Min_PAE_of_Interaction,36
Mean_PAE_of_Interaction,36


In [30]:
testgenes.to_csv("top_interaction_PAE.csv")